# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [30]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [31]:
file= "../Output/weather_data.csv"
weather_df= pd.read_csv(file)
weather_df

City  Cloudiness Country        Date  Humidity    Lat     Lng  \
0             bethel          20      US  1586645343        25  41.37  -73.41   
1              bluff          82      NZ  1586645276        62 -46.60  168.33   
2     vestmannaeyjar         100      IS  1586645543        88  63.44  -20.27   
3             bethel          20      US  1586645343        25  41.37  -73.41   
4              bluff          82      NZ  1586645276        62 -46.60  168.33   
..               ...         ...     ...         ...       ...    ...     ...   
570      chachapoyas          99      PE  1586646506        49  -6.23  -77.87   
571         smithers           5      CA  1586645986        26  54.78 -127.17   
572      sansepolcro           0      IT  1586646507        66  43.57   12.14   
573  santa fe do sul           3      BR  1586646507        65 -20.21  -50.93   
574     lawrenceburg          90      US  1586646507        26  38.04  -84.90   

     Max Temp  Wind Speed  
0       12.78        6.70  
1       15.56        1.79  
2        6.27        4.98  
3       12.78        6.70  
4       15.56        1.79  
..        ...         ...  
570     13.06        0.53  
571      7.00        1.00  
572     14.44        2.60  
573     21.22        1.96  
574     16.11        3.60  

[575 rows x 9 columns]

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [32]:
gmaps.configure(api_key=g_key)

In [33]:
humidity = weather_df["Humidity"]
locations = weather_df[["Lat", "Lng"]]
heatmap = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, max_intensity= max(humidity))
heatmap.add_layer(heat_layer)
heatmap

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [34]:
ideal_df = weather_df
ideal_df= ideal_df.loc[(ideal_df["Max Temp"] < 45) & (ideal_df["Max Temp"] > 20)]                                                                   
ideal_df= ideal_df.loc[ideal_df["Wind Speed"] < 35]
ideal_df= ideal_df.loc[ideal_df["Cloudiness"] < 10]

ideal_df.head()

City  Cloudiness Country        Date  Humidity    Lat     Lng  \
19     amahai           0      ID  1586645723        69  -3.33  128.92   
29  hithadhoo           5      MV  1586645550        67  -0.60   73.08   
39    deoband           5      IN  1586645725        36  29.70   77.68   
52       hilo           1      US  1586645363        62  19.73 -155.09   
53       tura           0      IN  1586645548        46  25.52   90.22   

    Max Temp  Wind Speed  
19     28.55        1.84  
29     29.09        3.94  
39     21.97        2.12  
52     29.00        3.60  
53     21.82        2.17

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [35]:
hotel_df= ideal_df

In [36]:
hotel_df.head()

City  Cloudiness Country        Date  Humidity    Lat     Lng  \
19     amahai           0      ID  1586645723        69  -3.33  128.92   
29  hithadhoo           5      MV  1586645550        67  -0.60   73.08   
39    deoband           5      IN  1586645725        36  29.70   77.68   
52       hilo           1      US  1586645363        62  19.73 -155.09   
53       tura           0      IN  1586645548        46  25.52   90.22   

    Max Temp  Wind Speed  
19     28.55        1.84  
29     29.09        3.94  
39     21.97        2.12  
52     29.00        3.60  
53     21.82        2.17

In [43]:
hotel_df['Hotel Name'] = ''
hotels = []
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    url= "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
    "location": f"{lat},{lng}",
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    
}
    response = requests.get(url, params=params).json()
    
    try:
        hotels.append(response['results'][0]['name'])
    
    except (KeyError, IndexError):
        hotels.append('No Hotel in 5km radius')

In [46]:
hotel_df["Hotel Name"] = hotels
hotel_df.head()
narrowed_city_df = hotel_df

In [47]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [48]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
heatmap.add_layer(markers)
# Display Map
heatmap

Figure(layout=FigureLayout(height='420px'))